In [1]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path

import gbd_mapping as gm
from vivarium import Artifact

from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
from get_draws.api import get_draws

!date
!whoami
!pwd

Wed Oct  1 06:18:35 PDT 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/scratch


In [7]:
pd.__version__

'1.5.3'

In [4]:
%cd /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/

/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers


In [31]:
!ls -halt results/model5.0/model_spec/2025_09_25_09_02_38/results/

total 106M
drwxrwsr-x 5 tylerdy IHME-Simulationscience 4.5K Sep 25 10:18 ..
drwxrwsr-x 2 tylerdy IHME-Simulationscience 3.0K Sep 25 09:36 .
-rw-rw-r-- 1 tylerdy IHME-Simulationscience 3.1M Sep 25 09:36 new_simulants.parquet
-rw-rw-r-- 1 tylerdy IHME-Simulationscience  15M Sep 25 09:36 person_time_alzheimers_disease_and_other_dementias.parquet
-rw-rw-r-- 1 tylerdy IHME-Simulationscience 4.6M Sep 25 09:36 deaths.parquet
-rw-rw-r-- 1 tylerdy IHME-Simulationscience 5.7M Sep 25 09:36 transition_count_alzheimers_disease_and_other_dementias.parquet
-rw-rw-r-- 1 tylerdy IHME-Simulationscience  51M Sep 25 09:36 ylds.parquet
-rw-rw-r-- 1 tylerdy IHME-Simulationscience  28M Sep 25 09:36 ylls.parquet


In [2]:
locations = [
            'united_states_of_america',
            #  'brazil',
            #  'china',
            #  'germany',
            #  'israel',
            #  'japan',
            #  'spain',
            #  'sweden',
            #  'taiwan_(province_of_china)',
            #  'united_kingdom',
            ]



results_dirs = [
    'results/model5.0/model_spec/2025_09_25_09_02_38/results/',
]

artifact_dirs = ['artifacts/model5.0/' + x + '.hdf' for x in locations]
artifact_dirs

['artifacts/model5.0/united_states_of_america.hdf']

In [5]:
%%time
test = pd.read_parquet(
    results_dirs[0] + 'deaths' + '.parquet',
    filters=[('artifact_path', '==', '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/' + artifact_dirs[0])]
)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570000 entries, 0 to 569999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   measure            570000 non-null  object  
 1   entity_type        570000 non-null  category
 2   entity             570000 non-null  category
 3   sub_entity         570000 non-null  category
 4   age_group          570000 non-null  category
 5   current_year       570000 non-null  category
 6   sex                570000 non-null  category
 7   artifact_path      570000 non-null  object  
 8   input_draw_number  570000 non-null  int64   
 9   scenario           570000 non-null  object  
 10  input_draw         570000 non-null  int64   
 11  random_seed        570000 non-null  int64   
 12  value              570000 non-null  float64 
dtypes: category(6), float64(1), int64(3), object(3)
memory usage: 33.7+ MB
CPU times: user 2.31 s, sys: 1.18 s, total: 3.49 s
Wall time: 1

In [16]:
print(test.memory_usage(deep=True).sum() / 1e6)
test.memory_usage(deep=True)

190.38932


Index                     132
measure              35910000
entity_type            570170
entity                 570258
sub_entity             570258
age_group              571530
current_year           576740
sex                    570232
artifact_path        95760000
input_draw_number     4560000
scenario             37050000
input_draw            4560000
random_seed           4560000
value                 4560000
dtype: int64

# Try converting a column to a Pyarrow backend

Hmm, it works, but it doesn't save much memory: 96 MB to 67 MB.

In [10]:
a = pd.Series(test.artifact_path, dtype='string[pyarrow]')
a

0         /mnt/team/simulation_science/pub/models/vivari...
1         /mnt/team/simulation_science/pub/models/vivari...
2         /mnt/team/simulation_science/pub/models/vivari...
3         /mnt/team/simulation_science/pub/models/vivari...
4         /mnt/team/simulation_science/pub/models/vivari...
                                ...                        
569995    /mnt/team/simulation_science/pub/models/vivari...
569996    /mnt/team/simulation_science/pub/models/vivari...
569997    /mnt/team/simulation_science/pub/models/vivari...
569998    /mnt/team/simulation_science/pub/models/vivari...
569999    /mnt/team/simulation_science/pub/models/vivari...
Name: artifact_path, Length: 570000, dtype: string

In [11]:
a.memory_usage(deep=True)

65550132

# Now try converting the column to Categorical

This saves lots of memory: 96 MB to 570 KB.

In [12]:
c = test.artifact_path.astype('category')
c

0         /mnt/team/simulation_science/pub/models/vivari...
1         /mnt/team/simulation_science/pub/models/vivari...
2         /mnt/team/simulation_science/pub/models/vivari...
3         /mnt/team/simulation_science/pub/models/vivari...
4         /mnt/team/simulation_science/pub/models/vivari...
                                ...                        
569995    /mnt/team/simulation_science/pub/models/vivari...
569996    /mnt/team/simulation_science/pub/models/vivari...
569997    /mnt/team/simulation_science/pub/models/vivari...
569998    /mnt/team/simulation_science/pub/models/vivari...
569999    /mnt/team/simulation_science/pub/models/vivari...
Name: artifact_path, Length: 570000, dtype: category
Categories (1, object): ['/mnt/team/simulation_science/pub/models/vivar...]

In [13]:
c.memory_usage(deep=True)

570408

In [14]:
a.unique()

<ArrowStringArray>
['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/united_states_of_america.hdf']
Length: 1, dtype: string

In [18]:
# pd.set_option("mode.dtype_backend", "pyarrow")
# Results in:
# OptionError: No such keys(s): 'mode.dtype_backend'
# since we're only using pandas 1.5.3

In [20]:
%%time
test_c = test.copy()

def convert_to_categorical(df):
    for col in df:
        if df[col].dtype not in ('float', 'category'):
            df[col] = df[col].astype('category')

convert_to_categorical(test_c)
test_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570000 entries, 0 to 569999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   measure            570000 non-null  category
 1   entity_type        570000 non-null  category
 2   entity             570000 non-null  category
 3   sub_entity         570000 non-null  category
 4   age_group          570000 non-null  category
 5   current_year       570000 non-null  category
 6   sex                570000 non-null  category
 7   artifact_path      570000 non-null  category
 8   input_draw_number  570000 non-null  category
 9   scenario           570000 non-null  category
 10  input_draw         570000 non-null  category
 11  random_seed        570000 non-null  category
 12  value              570000 non-null  float64 
dtypes: category(12), float64(1)
memory usage: 10.9 MB
CPU times: user 420 ms, sys: 46.8 ms, total: 467 ms
Wall time: 465 ms


In [23]:
m = test_c.memory_usage(deep=True)
print(m.sum() / 1e6)
m

11.411664


Index                    132
measure               570171
entity_type           570170
entity                570258
sub_entity            570258
age_group             571530
current_year          576740
sex                   570232
artifact_path         570276
input_draw_number     570756
scenario              570173
input_draw            570756
random_seed           570212
value                4560000
dtype: int64

# Try reading without filtering

It works! And it's not too slow. The deaths dataframe is 1.8 GB in memory.

In [24]:
%%time
deaths = pd.read_parquet(results_dirs[0] + 'deaths' + '.parquet')
deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5700000 entries, 0 to 5699999
Data columns (total 13 columns):
 #   Column             Dtype   
---  ------             -----   
 0   measure            object  
 1   entity_type        category
 2   entity             category
 3   sub_entity         category
 4   age_group          category
 5   current_year       category
 6   sex                category
 7   artifact_path      object  
 8   input_draw_number  int64   
 9   scenario           object  
 10  input_draw         int64   
 11  random_seed        int64   
 12  value              float64 
dtypes: category(6), float64(1), int64(3), object(3)
memory usage: 337.0+ MB
CPU times: user 2.54 s, sys: 852 ms, total: 3.39 s
Wall time: 1.18 s


In [25]:
m = deaths.memory_usage(deep=True)
print(m.sum() / 1e6)
m

1826.28932


Index                      132
measure              359100000
entity_type            5700170
entity                 5700258
sub_entity             5700258
age_group              5701530
current_year           5706740
sex                    5700232
artifact_path        880080000
input_draw_number     45600000
scenario             370500000
input_draw            45600000
random_seed           45600000
value                 45600000
dtype: int64

In [26]:
%%time
convert_to_categorical(deaths)
deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5700000 entries, 0 to 5699999
Data columns (total 13 columns):
 #   Column             Dtype   
---  ------             -----   
 0   measure            category
 1   entity_type        category
 2   entity             category
 3   sub_entity         category
 4   age_group          category
 5   current_year       category
 6   sex                category
 7   artifact_path      category
 8   input_draw_number  category
 9   scenario           category
 10  input_draw         category
 11  random_seed        category
 12  value              float64 
dtypes: category(12), float64(1)
memory usage: 108.7 MB
CPU times: user 2.72 s, sys: 541 ms, total: 3.26 s
Wall time: 3.25 s


In [27]:
m = deaths.memory_usage(deep=True)
print(m.sum() / 1e6)
m

114.013232


Index                     132
measure               5700171
entity_type           5700170
entity                5700258
sub_entity            5700258
age_group             5701530
current_year          5706740
sex                   5700232
artifact_path         5701844
input_draw_number     5700756
scenario              5700173
input_draw            5700756
random_seed           5700212
value                45600000
dtype: int64

In [28]:
deaths

,measure,entity_type,entity,sub_entity,age_group,current_year,sex,artifact_path,input_draw_number,scenario,input_draw,random_seed,value
0,deaths,cause,alzheimers_disease_state,alzheimers_disease_state,25_to_29,2025,Female,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.0
1,deaths,cause,alzheimers_disease_state,alzheimers_disease_state,25_to_29,2025,Male,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.0
2,deaths,cause,alzheimers_disease_state,alzheimers_disease_state,25_to_29,2026,Female,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.0
3,deaths,cause,alzheimers_disease_state,alzheimers_disease_state,25_to_29,2026,Male,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.0
4,deaths,cause,alzheimers_disease_state,alzheimers_disease_state,25_to_29,2027,Female,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5699995,deaths,cause,other_causes,other_causes,95_plus,2098,Male,/mnt/team/simulation_science/pub/models/vivari...,199,baseline,199,5616,70.0
5699996,deaths,cause,other_causes,other_causes,95_plus,2099,Female,/mnt/team/simulation_science/pub/models/vivari...,199,baseline,199,5616,532.0
5699997,deaths,cause,other_causes,other_causes,95_plus,2099,Male,/mnt/team/simulation_science/pub/models/vivari...,199,baseline,199,5616,60.0
5699998,deaths,cause,other_causes,other_causes,95_plus,2100,Female,/mnt/team/simulation_science/pub/models/vivari...,199,baseline,199,5616,503.0


In [30]:
deaths.artifact_path.unique().to_list()

['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/sweden.hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/germany.hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/japan.hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/spain.hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/united_kingdom.hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/taiwan_(province_of_china).hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/united_states_of_america.hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/china.hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/israel.hdf',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifact

# Try loading YLDs, which is the largest file at 51MB

The YLDs dataframe has 11.4 million rows. When read in, the dataframe is 3.6 GB in memory. After conversion to
Categoricals, it's only 228 MB.

In [32]:
%%time
ylds = pd.read_parquet(results_dirs[0] + 'ylds' + '.parquet')
ylds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11400000 entries, 0 to 11399999
Data columns (total 13 columns):
 #   Column             Dtype   
---  ------             -----   
 0   measure            object  
 1   entity_type        category
 2   entity             category
 3   sub_entity         category
 4   age_group          category
 5   current_year       category
 6   sex                category
 7   artifact_path      object  
 8   input_draw_number  int64   
 9   scenario           object  
 10  input_draw         int64   
 11  random_seed        int64   
 12  value              float64 
dtypes: category(6), float64(1), int64(3), object(3)
memory usage: 674.1+ MB
CPU times: user 5.71 s, sys: 1.61 s, total: 7.32 s
Wall time: 2.64 s


In [33]:
m = ylds.memory_usage(deep=True)
print(m.sum() / 1e6)
m

3629.769588


Index                       132
measure               695400000
entity_type            11400170
entity                 11400270
sub_entity             11400514
age_group              11401530
current_year           11406740
sex                    11400232
artifact_path        1760160000
input_draw_number      91200000
scenario              741000000
input_draw             91200000
random_seed            91200000
value                  91200000
dtype: int64

In [34]:
%%time
convert_to_categorical(ylds)
ylds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11400000 entries, 0 to 11399999
Data columns (total 13 columns):
 #   Column             Dtype   
---  ------             -----   
 0   measure            category
 1   entity_type        category
 2   entity             category
 3   sub_entity         category
 4   age_group          category
 5   current_year       category
 6   sex                category
 7   artifact_path      category
 8   input_draw_number  category
 9   scenario           category
 10  input_draw         category
 11  random_seed        category
 12  value              float64 
dtypes: category(12), float64(1)
memory usage: 217.4 MB
CPU times: user 3.66 s, sys: 841 ms, total: 4.5 s
Wall time: 4.49 s


In [35]:
m = ylds.memory_usage(deep=True)
print(m.sum() / 1e6)
m

228.013498


Index                     132
measure              11400169
entity_type          11400170
entity               11400270
sub_entity           11400514
age_group            11401530
current_year         11406740
sex                  11400232
artifact_path        11401844
input_draw_number    11400756
scenario             11400173
input_draw           11400756
random_seed          11400212
value                91200000
dtype: int64

# Now try aggregating over random seeds

This reduces the number of rows by a factor of 5 (since there are 5
random seeds), from 11.4 million to 2.28 million. The memory usage
decreases from 228 MB to 43 MB, also approximately a factor of 5.

Note that we need to pass `observed=True` to the `DataFrame.groupby()`,
or else the size of the result will explode because of unused categories
in the Categoricals.

In [36]:
ylds

,measure,entity_type,entity,sub_entity,age_group,current_year,sex,artifact_path,input_draw_number,scenario,input_draw,random_seed,value
0,ylds,cause,alzheimers_disease_and_other_dementias,alzheimers_blood_based_biomarker_state,25_to_29,2025,Female,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.000000
1,ylds,cause,alzheimers_disease_and_other_dementias,alzheimers_mild_cognitive_impairment_state,25_to_29,2025,Female,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.000000
2,ylds,cause,alzheimers_disease_and_other_dementias,alzheimers_disease_state,25_to_29,2025,Female,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.000000
3,ylds,cause,all_causes,all_causes,25_to_29,2025,Female,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.000000
4,ylds,cause,alzheimers_disease_and_other_dementias,alzheimers_blood_based_biomarker_state,25_to_29,2025,Male,/mnt/team/simulation_science/pub/models/vivari...,499,baseline,499,2284,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11399995,ylds,cause,all_causes,all_causes,95_plus,2100,Female,/mnt/team/simulation_science/pub/models/vivari...,199,baseline,199,5616,475.705255
11399996,ylds,cause,alzheimers_disease_and_other_dementias,alzheimers_blood_based_biomarker_state,95_plus,2100,Male,/mnt/team/simulation_science/pub/models/vivari...,199,baseline,199,5616,0.000000
11399997,ylds,cause,alzheimers_disease_and_other_dementias,alzheimers_mild_cognitive_impairment_state,95_plus,2100,Male,/mnt/team/simulation_science/pub/models/vivari...,199,baseline,199,5616,0.622612
11399998,ylds,cause,alzheimers_disease_and_other_dementias,alzheimers_disease_state,95_plus,2100,Male,/mnt/team/simulation_science/pub/models/vivari...,199,baseline,199,5616,41.077482


In [56]:
%%time
groupby_cols = ylds.columns.difference(['random_seed', 'value']).to_list()
temp = ylds.groupby(groupby_cols, observed=True)['value'].sum()
temp

CPU times: user 4.91 s, sys: 1.29 s, total: 6.2 s
Wall time: 6.2 s


age_group  artifact_path                                                                                  current_year  entity                                  entity_type  input_draw  input_draw_number  measure  scenario  sex     sub_entity                                
25_to_29   /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model5.0/sweden.hdf  2025          alzheimers_disease_and_other_dementias  cause        499         499                ylds     baseline  Female  alzheimers_blood_based_biomarker_state           0.000000
                                                                                                                                                                                                                                       alzheimers_mild_cognitive_impairment_state       0.000000
                                                                                                                                                    

In [57]:
2280000 / 1e6

2.28

In [41]:
11400000 / 1e6

11.4

In [62]:
11400000 / 2280000

5.0

In [58]:
m = temp.memory_usage(deep=True)
print(m / 1e6, 'MB')
m

43.333961 MB


43333961

In [59]:
temp.index.unique('sub_entity')

CategoricalIndex(['alzheimers_blood_based_biomarker_state',
                  'alzheimers_mild_cognitive_impairment_state',
                  'alzheimers_disease_state', 'all_causes'],
                 categories=['all_causes', 'alzheimers_blood_based_biomarker_state', 'alzheimers_disease_state', 'alzheimers_mild_cognitive_impairment_state'], ordered=False, dtype='category', name='sub_entity')

In [61]:
temp.index.unique('entity')

CategoricalIndex(['alzheimers_disease_and_other_dementias', 'all_causes'], categories=['all_causes', 'alzheimers_disease_and_other_dementias'], ordered=False, dtype='category', name='entity')

In [60]:
temp.index.unique('age_group')

CategoricalIndex(['25_to_29', '30_to_34', '35_to_39', '40_to_44', '45_to_49',
                  '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74',
                  '75_to_79', '80_to_84', '85_to_89', '90_to_94', '95_plus'],
                 categories=['25_to_29', '30_to_34', '35_to_39', '40_to_44', ..., '80_to_84', '85_to_89', '90_to_94', '95_plus'], ordered=False, dtype='category', name='age_group')

In [48]:
temp.index.names

FrozenList(['age_group', 'artifact_path', 'current_year', 'entity', 'entity_type', 'input_draw', 'input_draw_number', 'measure', 'scenario', 'sex', 'sub_entity'])

In [54]:
temp.index.get_level_values('age_group')

CategoricalIndex(['25_to_29', '25_to_29', '25_to_29', '25_to_29', '25_to_29',
                  '25_to_29', '25_to_29', '25_to_29', '25_to_29', '25_to_29',
                  ...
                  '95_plus', '95_plus', '95_plus', '95_plus', '95_plus',
                  '95_plus', '95_plus', '95_plus', '95_plus', '95_plus'],
                 categories=['25_to_29', '30_to_34', '35_to_39', '40_to_44', ..., '80_to_84', '85_to_89', '90_to_94', '95_plus'], ordered=False, dtype='category', name='age_group', length=114000000)

In [55]:
temp.isna().any()

False

In [63]:
228 / 43.3

5.265588914549654